Import the libraries we will be using. Right now that is beautiful soup.

In [14]:
from bs4 import BeautifulSoup
import urllib
import json
import re

Grab the website

In [2]:
nrmp_page = urllib.urlopen("http://www.nrmp.org/participating-fellowships/").read()
nrmp_soup = BeautifulSoup(nrmp_page)
print type(nrmp_soup)

<class 'bs4.BeautifulSoup'>


In [3]:
print nrmp_soup.prettify()[0:1000]

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=1150" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="http://www.nrmp.org/xmlrpc.php" rel="pingback"/>
  <title>
   Participating Fellowships |
  </title>
  <link href="http://www.nrmp.org/participating-fellowships/feed/" rel="alternate" title="The Match, National Resident Matching Program » Participating Fellowships Comments Feed" type="application/rss+xml"/>
  <script type="text/javascript">
   window._wpemojiSettings = {"baseUrl":"http:\/\/s.w.org\/images\/core\/emoji\/72x72\/","ext":".png","source":{"concatemoji":"http:\/\/www.nrmp.org\/wp-includes\/js\/wp-emoji-release.min.js?ver=4.3"}};
			!function(a,b,c){function d(a){var c=b.createElement("canvas"),d=c.getContext&&c.getContext("2d");return d&&d.fillText?(d.textBaseline="top",d.font="600 32px Arial","flag"===a?(d.fillText(String.fromCharCode(55356,56812,55356,56807),0,0),c.toDataURL().length>3e3):(d

I searched on the NRMP page for the parent class that encompases all of the fellowships. This had the class `main-box`.

In [4]:
nrmp_soup.find_all(class_="main-box")

[<div class="main-box">
 <h2>FELLOWSHIP MATCHES</h2>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/abdominal-transplant-surgery-match/"><span class="title">Abdominal Transplant Surgery Match</span></a>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/adolescent-medicine-match/"><span class="title">Adolescent Medicine Match</span></a>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/allergy-immunology-match/"><span class="title">Allergy / Immunology Match</span></a>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/anesthesiology-match/"><span class="title">Anesthesiology Match</span></a>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/colon-and-rectal-surgery-match/"><span class="title">Colon and Rectal Surgery Match</span></a>
 <a class="individual-boxSMALL" href="http://www.nrmp.org/fellowships/combined-musculoskeletal-hand-surgery-matching-program-cmmp-match/"><span class="tit

All of the fellowships have "span" tags and the class "title" so we make a little for-loop that will print out the text of each span within the "main-box" class if it has a tag of "span"

In [44]:
for title in nrmp_soup.find_all(class_="main-box"):
    title_descendants = title.descendants
    for d in title_descendants:
        if d.name == "span":
            print d.text.split(" Match")[0]

Abdominal Transplant Surgery
Adolescent Medicine
Allergy / Immunology
Anesthesiology
Colon and Rectal Surgery
Combined Musculoskeletal – Hand Surgery
Female Pelvic Medicine and Reconstructive Surgery
Geriatric Medicine
Headache Medicine
Hospice and Palliative Medicine
Laryngology
Medical Genetics
Medical Specialties
Medical Toxicology
Obstetrics / Gynecology
Pediatric Rehabilitation Medicine
Pediatric Specialties Fall
Pediatric Specialties Spring
Pediatric Surgery
Primary Care Sports Medicine
Psychiatry
Radiology
Sleep Medicine
Spinal Cord Injury Medicine
Surgical Critical Care
Surgical Oncology
Thoracic Surgery
Vascular Neurology
Vascular Surgery


The goal is to get the data into a format that D3 understands, which is json. The way the json looks is a dictionary with list of dictionaries, which looks like the following for the nodes:

`{"nodes":[{"name":'Abdominal Transplant Surgery Match'},...`

I also had to convert the text to utf8 and then to string to get rid of the 'u' that was accompanying the text 

`str(d.text.encode('utf8'))`

In [107]:
nodes = []
temp_dict = {}
for title in nrmp_soup.find_all(class_="main-box"):
    title_descendants = title.descendants
    for d in title_descendants:
        if d.name == "span":
            temp_dict["name"] = str(d.text.split(" Match")[0].encode('utf8')).replace(' \xe2\x80\x93 ', ' ').replace(' / ', '/')
            nodes.append(temp_dict.copy())

In [108]:
nodes

[{'name': 'Abdominal Transplant Surgery'},
 {'name': 'Adolescent Medicine'},
 {'name': 'Allergy/Immunology'},
 {'name': 'Anesthesiology'},
 {'name': 'Colon and Rectal Surgery'},
 {'name': 'Combined Musculoskeletal Hand Surgery'},
 {'name': 'Female Pelvic Medicine and Reconstructive Surgery'},
 {'name': 'Geriatric Medicine'},
 {'name': 'Headache Medicine'},
 {'name': 'Hospice and Palliative Medicine'},
 {'name': 'Laryngology'},
 {'name': 'Medical Genetics'},
 {'name': 'Medical Specialties'},
 {'name': 'Medical Toxicology'},
 {'name': 'Obstetrics/Gynecology'},
 {'name': 'Pediatric Rehabilitation Medicine'},
 {'name': 'Pediatric Specialties Fall'},
 {'name': 'Pediatric Specialties Spring'},
 {'name': 'Pediatric Surgery'},
 {'name': 'Primary Care Sports Medicine'},
 {'name': 'Psychiatry'},
 {'name': 'Radiology'},
 {'name': 'Sleep Medicine'},
 {'name': 'Spinal Cord Injury Medicine'},
 {'name': 'Surgical Critical Care'},
 {'name': 'Surgical Oncology'},
 {'name': 'Thoracic Surgery'},
 {'name'

There are 29 felloships listed. I don't want to go each by hand and extract the pertinent information. Luckily, the page structure is consistent throughout:

`http://www.nrmp.org/fellowships/[name of fellowhip]/`

for example: http://www.nrmp.org/fellowships/adolescent-medicine-match/

Luckily, we can just pull the `href` text betwen the `a` tags to get a list of links, like so:

### Get list of links for fellowship details

In [48]:
# fellowships_lol = [name.values() for name in nodes] # returns list of lists
# fellowships = [name for sublist in fellowships_lol for name in sublist] # flattens list of lists
# fellowships

fellowship_links = []
for title in nrmp_soup.find_all(class_="main-box"):
    for a in title.find_all('a', href=True):
        fellowship_links.append(a['href'])
print fellowship_links

['http://www.nrmp.org/fellowships/abdominal-transplant-surgery-match/', 'http://www.nrmp.org/fellowships/adolescent-medicine-match/', 'http://www.nrmp.org/fellowships/allergy-immunology-match/', 'http://www.nrmp.org/fellowships/anesthesiology-match/', 'http://www.nrmp.org/fellowships/colon-and-rectal-surgery-match/', 'http://www.nrmp.org/fellowships/combined-musculoskeletal-hand-surgery-matching-program-cmmp-match/', 'http://www.nrmp.org/fellowships/female-pelvic-medicine-and-reconstructive-surgery-match/', 'http://www.nrmp.org/fellowships/geriatric-medicine-match/', 'http://www.nrmp.org/fellowships/headache-medicine/', 'http://www.nrmp.org/fellowships/hospice-palliative-medicine/', 'http://www.nrmp.org/fellowships/laryngology-match/', 'http://www.nrmp.org/fellowships/medical-genetics-match/', 'http://www.nrmp.org/fellowships/medical-specialties-matching-program/', 'http://www.nrmp.org/fellowships/medical-toxicology-match/', 'http://www.nrmp.org/fellowships/obstetricsgynecology-match/'

### Test script for pulling details from fellowship page

In [80]:
adolescent_med = urllib.urlopen("http://www.nrmp.org/fellowships/abdominal-transplant-surgery-match/").read()
adolescent_med_soup = BeautifulSoup(adolescent_med)

### Pull relevant information

This step is a bit mor involved. Here's what we want to do:

1. Go to each page in the list
2. Pull residency that must be completed
3. Add the residency that must be completed to the list of nodes
4. Convert `nodes` from list to dict with `nodes = dict({"nodes": nodes})`

In [55]:
adolescent_med_soup.prettify()[0:1000] #inspect

u'<!DOCTYPE html>\n<html lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=1150" name="viewport"/>\n  <link href="http://gmpg.org/xfn/11" rel="profile"/>\n  <link href="http://www.nrmp.org/xmlrpc.php" rel="pingback"/>\n  <title>\n   Adolescent Medicine Match |\n  </title>\n  <script type="text/javascript">\n   window._wpemojiSettings = {"baseUrl":"http:\\/\\/s.w.org\\/images\\/core\\/emoji\\/72x72\\/","ext":".png","source":{"concatemoji":"http:\\/\\/www.nrmp.org\\/wp-includes\\/js\\/wp-emoji-release.min.js?ver=4.3"}};\n\t\t\t!function(a,b,c){function d(a){var c=b.createElement("canvas"),d=c.getContext&&c.getContext("2d");return d&&d.fillText?(d.textBaseline="top",d.font="600 32px Arial","flag"===a?(d.fillText(String.fromCharCode(55356,56812,55356,56807),0,0),c.toDataURL().length>3e3):(d.fillText(String.fromCharCode(55357,56835),0,0),0!==d.getImageData(16,16,1,1).data[0])):!1}function e(a){var c=b.createElement("script");c.src=a,c.type="text/javascript",b.getElem

In [56]:
# for chunk in adolescent_med_soup.find_all(class_="match-row"):
#     residency = str(chunk.find("li").string).split('in ')[1:]
#     residency_text = ', '.join(residency)
#     residency_list = residency_text.split(', ')

In [98]:
for chunk in adolescent_med_soup.find_all(class_="match-row"):
    residency = str(chunk.find("li").string).split('in ')[1:]
    residency_text = ', '.join(residency)
    residency_list = residency_text.split(' or ')
#     pattern = re.compile("(or)\W", re.I)
#     residency_list = map(lambda residency: pattern.sub("", residency),  residency_list)
print residency_list

['General Surgery', 'Urology']


Now that we have a clean list of residencies, we need to put them in the right json format like we did the fellowships:

[{'name': 'Abdominal Transplant Surgery Match'},
 {'name': 'Adolescent Medicine Match'},
 {'name': 'Allergy/Immunology Match'},
 {'name': 'Anesthesiology Match'},
 {'name': 'Colon and Rectal Surgery Match'},
 {'name': 'Combined Musculoskeletal Hand Surgery Matching Program Match'},
 {'name': 'Female Pelvic Medicine and Reconstructive Surgery Match'},
 {'name': 'Geriatric Medicine Match'},
 {'name': 'Headache Medicine'},
 {'name': 'Hospice and Palliative Medicine'},
 {'name': 'Laryngology Match'},
 {'name': 'Medical Genetics Match'},
 {'name': 'Medical Specialties Matching Program'},
 {'name': 'Medical Toxicology Match'},
 {'name': 'Obstetrics/Gynecology Match'},
 {'name': 'Pediatric Rehabilitation Medicine Match'},
 {'name': 'Pediatric Specialties Fall Match'},
 {'name': 'Pediatric Specialties Spring Match'},
 {'name': 'Pediatric Surgery Match'},
 {'name': 'Primary Care Sports Medicine Match'},
 {'name': 'Psychiatry Match'},
 {'name': 'Radiology Match'},
 {'name': 'Sleep Medicine Match

In [109]:
for link in fellowship_links:
    try:
        fellowship_link = urllib.urlopen(link).read()
        fellowship_link_soup = BeautifulSoup(fellowship_link)
        for chunk in fellowship_link_soup.find_all(class_="match-row"):
            residency = str(chunk.find("li").string).split('in ')[1:]
            residency_text = ', '.join(residency)
            if "," in residency_text:
                residency_list = residency_text.split(', ')
                pattern = re.compile("(or)\W", re.I)
                residency_list = map(lambda residency: pattern.sub("", residency),  residency_list)
            else:
                residency_list = re.split((' or '), residency_text)
        for residency in residency_list:
            temp_dict["name"] = residency.title()
            if not any(d['name'] == residency for d in nodes):
                nodes.append(temp_dict.copy())
            else:
                pass
    except:
        print "%s did not work" %(link)
nodes

http://www.nrmp.org/fellowships/allergy-immunology-match/ did not work
http://www.nrmp.org/fellowships/geriatric-medicine-match/ did not work
http://www.nrmp.org/fellowships/medical-genetics-match/ did not work


[{'name': 'Abdominal Transplant Surgery'},
 {'name': 'Adolescent Medicine'},
 {'name': 'Allergy/Immunology'},
 {'name': 'Anesthesiology'},
 {'name': 'Colon and Rectal Surgery'},
 {'name': 'Combined Musculoskeletal Hand Surgery'},
 {'name': 'Female Pelvic Medicine and Reconstructive Surgery'},
 {'name': 'Geriatric Medicine'},
 {'name': 'Headache Medicine'},
 {'name': 'Hospice and Palliative Medicine'},
 {'name': 'Laryngology'},
 {'name': 'Medical Genetics'},
 {'name': 'Medical Specialties'},
 {'name': 'Medical Toxicology'},
 {'name': 'Obstetrics/Gynecology'},
 {'name': 'Pediatric Rehabilitation Medicine'},
 {'name': 'Pediatric Specialties Fall'},
 {'name': 'Pediatric Specialties Spring'},
 {'name': 'Pediatric Surgery'},
 {'name': 'Primary Care Sports Medicine'},
 {'name': 'Psychiatry'},
 {'name': 'Radiology'},
 {'name': 'Sleep Medicine'},
 {'name': 'Spinal Cord Injury Medicine'},
 {'name': 'Surgical Critical Care'},
 {'name': 'Surgical Oncology'},
 {'name': 'Thoracic Surgery'},
 {'name'

Now we have a clean set of nodes.

Now that we can do this scraping for one fellowship/residency, we need to do it for all of them:

In [ ]:
for link in fellowship_links:
    fellowship_link = urllib.urlopen(link).read()
    fellowship_link_soup = BeautifulSoup(adolescent_med)
    for chunk in fellowship_link_soup.find_all(class_="match-row"):
        residency = str(chunk.find("li").string).split('in ')[1:]
        residency_text = ', '.join(residency)
        residency_list = residency_text.split(', ')
        pattern = re.compile("(or)\W", re.I)
        residency_list = map(lambda residency: pattern.sub("", residency),  residency_list)
        for residency in residency_list:
            temp_dict["name"] = residency
            for i in range(len(nodes)):
                if residency not in nodes[i].values():
                    nodes.append(temp_dict.copy())
                else:
                    pass
nodes

In [36]:
'Adolescent Medicine Match' in nodes[1].values()

True

In [37]:
nodes

[{'name': 'Abdominal Transplant Surgery Match'},
 {'name': 'Adolescent Medicine Match'},
 {'name': 'Allergy/Immunology Match'},
 {'name': 'Anesthesiology Match'},
 {'name': 'Colon and Rectal Surgery Match'},
 {'name': 'Combined Musculoskeletal Hand Surgery Matching Program Match'},
 {'name': 'Female Pelvic Medicine and Reconstructive Surgery Match'},
 {'name': 'Geriatric Medicine Match'},
 {'name': 'Headache Medicine'},
 {'name': 'Hospice and Palliative Medicine'},
 {'name': 'Laryngology Match'},
 {'name': 'Medical Genetics Match'},
 {'name': 'Medical Specialties Matching Program'},
 {'name': 'Medical Toxicology Match'},
 {'name': 'Obstetrics/Gynecology Match'},
 {'name': 'Pediatric Rehabilitation Medicine Match'},
 {'name': 'Pediatric Specialties Fall Match'},
 {'name': 'Pediatric Specialties Spring Match'},
 {'name': 'Pediatric Surgery Match'},
 {'name': 'Primary Care Sports Medicine Match'},
 {'name': 'Psychiatry Match'},
 {'name': 'Radiology Match'},
 {'name': 'Sleep Medicine Match